# AdaBoost with MNIST Dataset

## 1. Import Library

In [1]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## 2. Load MNIST dataset

In [2]:
# Load MNIST dataset
mnist = fetch_openml('mnist_784', version=1, parser="auto")
X = mnist['data'].values
y = mnist['target'].astype(np.int32).values

# Binarize the labels (convert to binary classification for simplicity)
y = (y == 0).astype(np.int32)  # Let's classify '0' vs 'not 0'

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
# Test with 100 training samples and 10 testing samples
X_train = X_train[:100]
X_test = X_test[:10]
y_train = y_train[:100]
y_test = y_test[:10]

## 3. AdaBoost Model

In [4]:
class DecisionStump:
    def __init__(self):
        self.polarity = 1
        self.feature_index = None
        self.threshold = None
        self.alpha = None

    def predict(self, X):
        n_samples = X.shape[0]
        X_column = X[:, self.feature_index]
        predictions = np.ones(n_samples)
        if self.polarity == 1:
            predictions[X_column < self.threshold] = -1
        else:
            predictions[X_column > self.threshold] = -1
        return predictions

class AdaBoost:
    def __init__(self, n_clf=5):
        self.n_clf = n_clf

    def fit(self, X, y):
        n_samples, n_features = X.shape
        y = np.where(y == 0, -1, 1)

        # Initialize weights
        w = np.full(n_samples, (1 / n_samples))

        self.clfs = []

        for _ in range(self.n_clf):
            clf = DecisionStump()
            min_error = float('inf')

            # Loop over all features
            for feature_i in range(n_features):
                X_column = X[:, feature_i]
                thresholds = np.unique(X_column)

                for threshold in thresholds:
                    p = 1
                    predictions = np.ones(n_samples)
                    predictions[X_column < threshold] = -1

                    # Calculate error
                    error = np.sum(w * (predictions != y))

                    # If the error is greater than 50%, flip the polarity
                    if error > 0.5:
                        error = 1 - error
                        p = -1

                    # Store the best stump
                    if error < min_error:
                        clf.polarity = p
                        clf.threshold = threshold
                        clf.feature_index = feature_i
                        min_error = error

            # Calculate alpha
            clf.alpha = 0.5 * np.log((1 - min_error) / (min_error + 1e-10))

            # Update weights
            predictions = clf.predict(X)
            w *= np.exp(-clf.alpha * y * predictions)
            w /= np.sum(w)

            self.clfs.append(clf)

    def predict(self, X):
        clf_preds = np.zeros(X.shape[0])
        for clf in self.clfs:
            clf_preds += clf.alpha * clf.predict(X)
        y_pred = np.sign(clf_preds)
        return np.where(y_pred == -1, 0, 1)

## 4. Inference & Evaluate

In [5]:
# Train and evaluate the AdaBoost classifier
model = AdaBoost(n_clf=10)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

Accuracy: 1.0
